In [ ]:
!pip install -Uqq fastbook nbdev ipywidgets

In [ ]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *
import fastai
print(f'fastai version: {fastai.__version__}')

BASE_DIR='/content/gdrive/MyDrive/Colab Notebooks'
!ln -sf '/content/gdrive/MyDrive/Colab Notebooks' ~/.fastai

In [ ]:
path = untar_data(URLs.PETS)
# !ls -l ~/.fastai/data/oxf*/images | head
path , len((path/'images').ls()), len(get_image_files(path/'images'))

In [ ]:
def label_func(fname):
  return'_'.join(fname.name.split('_')[:-1])

db = DataBlock(blocks=(ImageBlock, CategoryBlock),
               get_items = get_image_files,
               splitter = RandomSplitter(valid_pct=0.2, seed=42),
               get_y = label_func,
               item_tfms=Resize(446),
               batch_tfms=aug_transforms(size=244, min_scale=0.75))
# db.summary(path/'images')
dls = db.dataloaders(path/'images')

In [ ]:
len(dls.train), len(dls.train_ds), len(dls.valid), len(dls.valid_ds)
x, y = dls.one_batch()
len(x), len(y), x.shape, y.shape

# Learner.save and Learner.load use the default path based on following fields
# learn.path, learn.model_dir

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)

## Load the model & weights from the file if they've been saved before
learn.load(file=Path(f'{BASE_DIR}/saves/oxford-iiit-pet'))

## Fine tune the model if no saves are available to load
#learn.fine_tune(2)

## Save the model & weights so we can load it later without incurring processing costs again and again
#learn.save(file=Path(f'{BASE_DIR}/saves/oxford-iiit-pet'))

In [ ]:
## The Learning Rate Finder
learn2 = vision_learner(dls, resnet34, metrics=error_rate)
learn2.fine_tune(1, base_lr=0.1)

In [ ]:
learn2.save(file=Path(f'{BASE_DIR}/saves/oxford-iiit-pet-lr-find'))
# learn2.load(file=Path(f'{BASE_DIR}/saves/oxford-iiit-pet-lr-find'))


In [ ]:
learn2 = cnn_learner(dls, resnet34, metrics=error_rate)
lr_finder = learn2.lr_find()


In [ ]:

lr_min = suggested_lrs.lr_min
lr_steep = suggested_lrs.lr_steep
print(f'min: {lr_min}, steep: {lr_steep}')

